In [1]:
import os
from dotenv import load_dotenv
import dask.dataframe as dd
from src.features.csv_to_parquet import STORE_DTYPES

load_dotenv()


True

In [2]:
ddf = dd.read_csv(
    "s3://food-delivery-crawler/data-interim/processed/*/store/**/*-store.csv",
    storage_options={"client_kwargs": {"endpoint_url": os.environ["S3_ENDPOINT"]}},
    dtype=STORE_DTYPES,
    include_path_column="filePath",
)


In [3]:
relevant_columns = [
    "name",
    "id",
    "cityId",
    "zoneId",
    "completeAddress",
    "filePath",
]
ddf = ddf.rename(
    columns={
        "location_cityId": "cityId",
        "location_zoneId": "zoneId",
        "location_address_address1": "completeAddress",
    }
)[relevant_columns]
store_ddf = (
    ddf.filePath.str.removesuffix("-store.csv")
    .str.split("/", expand=True, n=9)[[5, 6, 9]]
    .rename(columns={5: "city", 6: "zone", 9: "store"})
)

output_ddf = ddf.assign(city=store_ddf.city, zone=store_ddf.zone, store=store_ddf.store)
output_ddf = output_ddf.repartition(partition_size="1MB")


In [4]:
output_ddf.head()

,name,id,cityId,zoneId,completeAddress,filePath,city,zone,store
0,Happy Pizza,414349,235,1282,"Via Reich 62, Bergamo, 24020",food-delivery-crawler/data-interim/processed/d...,bergamo,alzano-lombardo,happy-pizza-bergamo
0,Pony Burger,276784,235,1282,"Via Agostino Gemelli, 17, Bergamo, 24124",food-delivery-crawler/data-interim/processed/d...,bergamo,alzano-lombardo,pony-burger
0,Mada Poke,201423,235,1282,"Via Andrea Fantoni 12, Bergamo, 24121",food-delivery-crawler/data-interim/processed/d...,bergamo,centro-storico,bar-mada
0,Bobino,281527,235,1282,"Piazza della Libertà 9,, Bergamo, 24121",food-delivery-crawler/data-interim/processed/d...,bergamo,centro-storico,bobino-bergamo
0,Burger King,218187,235,1282,"Viale Giulio Cesare, 18, Bergamo, 24124",food-delivery-crawler/data-interim/processed/d...,bergamo,centro-storico,burger-king-bg-giulio-cesare-27955
